In [2]:
import pandas as pd 
import numpy as np 
import tabula

Extracting household-level data from pdf. We do this page by page.

In [3]:
pdf_file = 'Cebu.Survey.Report[final].pdf'
df1 = tabula.read_pdf(pdf_file, pages = "21")
df2 = tabula.read_pdf(pdf_file, pages = "22")
df3 = tabula.read_pdf(pdf_file, pages = "23")

Got stderr: Apr 29, 2021 4:46:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 29, 2021 4:46:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 29, 2021 4:46:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 29, 2021 4:46:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Apr 29, 2021 4:46:08 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Apr 29, 2021 4:46:11 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [ ]:
def preprocess(df):
    

In [ ]:
tmp  = df2[2]
# remove "unnamed columns"
cols = tmp.columns[~tmp.columns.str.contains('Unnamed')]
tmp =tmp[cols].dropna()
tmp[tmp.iloc[:,0].str.contains('H/')]

Reconstructing the table

In [ ]:
d1[]

In [46]:
# remove second column strings that do not have H/H
# get relevant df only


,/H/12,5,2,1.,1.25,0,2.75,7.5,1.0,"1,070"
0,/H/13,6.0,1.9,0.0,2.00,0.0,0.75,5.1,0.7,740
1,/H/14,2.0,2.3,0.1,1.15,0.0,0.00,3.6,0.5,520
2,/H/15,1.0,1.7,0.6,0.65,0.2,0.75,4.0,0.5,580
3,/H/16,7.0,7.7,0.2,0.80,0.0,0.80,9.6,1.3,"1,370"
4,/H/17,4.0,1.2,0.8,0.50,0.0,0.85,3.4,0.4,490
6,/H/19,2.0,1.5,0.7,1.25,0.0,0.25,3.7,0.5,540
7,/H/20,8.0,6.2,0.7,0.50,0.0,2.75,10.2,1.4,"1,460"
11,/H/1,1.0,2.5,0.2,0.25,0.7,4.25,8.0,1.1,"1,140"
12,/H/2,2.0,3.5,0.7,0.90,0.0,4.75,9.9,1.4,"1,410"
13,/H/3,1.0,3.3,0.2,1.75,0.2,1.25,6.8,0.9,980


In [19]:
df1[2].columns

Index(['Unnamed: 0', 'Capital Site', 'Unnamed: 1', 'Unnamed: 2', '/H/members',
       'Unnamed: 3', 'Unnamed: 4', 'Organi', 'Unnamed: 5', 'Unnamed: 6',
       'Paper', 'Unnamed: 7', 'Unnamed: 8', 'lastic', 'Unnamed: 9',
       'Unnamed: 10', 'eta', 'Unnamed: 11', 'Unnamed: 12', 'thers',
       'Unnamed: 13', 'Unnamed: 14', 'otal (Kg)', 'Unnamed: 15', 'Unnamed: 16',
       'v.Kg/per Day', 'Unnamed: 17', 'Unnamed: 18', 'v.g/per Day',
       'Unnamed: 19'],
      dtype='object')